# Create Train/Validation/Test Splits (Standalone)

This notebook creates stratified train/validation/test splits from the real dataset index.
**This is a standalone version that includes all necessary code and doesn't require external files.**

## Purpose
- Load real dataset index or create dummy data
- Perform stratified splitting by wave type and direction
- Ensure balanced representation across splits
- Analyze split statistics and validate quality
- Generate comprehensive visualizations

In [ ]:
# Install required packages if not available
import subprocess
import sys

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
required_packages = ['pandas', 'matplotlib', 'seaborn', 'numpy', 'scikit-learn']
for package in required_packages:
    install_if_missing(package)

print("✓ All required packages are available")

In [ ]:
import os
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict, Counter
from pathlib import Path
from typing import Dict, List, Any
from scipy.stats import ks_2samp, chi2_contingency

print("✓ All libraries imported successfully")

## Utility Functions

In [ ]:
def set_seed(seed: int) -> None:
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)


def ensure_dir(path: str) -> None:
    """Create directory if it doesn't exist."""
    if path:
        os.makedirs(path, exist_ok=True)


def read_jsonl(path: str) -> List[Dict[str, Any]]:
    """Read JSONL file and return list of records."""
    items = []
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line_num, line in enumerate(f, 1):
                line = line.strip()
                if line:
                    try:
                        items.append(json.loads(line))
                    except json.JSONDecodeError as e:
                        print(f"Warning: Invalid JSON on line {line_num}: {e}")
    except FileNotFoundError:
        print(f"Warning: File not found: {path}")
    return items


def write_jsonl(items: List[Dict[str, Any]], path: str) -> None:
    """Write list of records to JSONL file."""
    ensure_dir(os.path.dirname(path))
    with open(path, "w", encoding="utf-8") as f:
        for r in items:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("✓ Utility functions defined")

## Configuration

In [ ]:
# Configuration - modify these paths as needed
CONFIG = {
    "real_index": "data/processed/real_index.jsonl",
    "out_dir": "data/processed/splits",
    "train_ratio": 0.8,
    "val_ratio": 0.1,
    "test_ratio": 0.1,  # Remaining after train and val
    "seed": 42,
    "create_dummy_if_missing": True,
    "dummy_samples": 150
}

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

print(f"\nSplit ratios - Train: {CONFIG['train_ratio']}, Val: {CONFIG['val_ratio']}, Test: {CONFIG['test_ratio']}")
print(f"Random seed: {CONFIG['seed']}")

## Dummy Data Generation

In [ ]:
def create_dummy_dataset(n_samples: int = 150) -> List[Dict[str, Any]]:
    """Create dummy dataset for demonstration when real data is not available."""
    
    wave_types = ["beach_break", "reef_break", "point_break", "closeout", "a_frame"]
    directions = ["left", "right", "both"]
    confidences = ["high", "medium", "low"]
    
    items = []
    
    for i in range(n_samples):
        # Create realistic wave parameters with some correlations
        wave_type = np.random.choice(wave_types)
        direction = np.random.choice(directions)
        
        # Height varies by wave type
        if wave_type == "closeout":
            height = float(np.random.uniform(0.3, 1.5))
        elif wave_type == "reef_break":
            height = float(np.random.uniform(1.0, 3.0))
        elif wave_type == "point_break":
            height = float(np.random.uniform(0.8, 2.5))
        else:
            height = float(np.random.uniform(0.5, 2.2))
        
        # Confidence correlates with wave type complexity
        if wave_type in ["closeout", "beach_break"]:
            confidence = np.random.choice(confidences, p=[0.6, 0.3, 0.1])
        else:
            confidence = np.random.choice(confidences, p=[0.4, 0.4, 0.2])
        
        items.append({
            "image_path": f"data/dummy/images/wave_{i:03d}.jpg",
            "height_meters": round(height, 2),
            "wave_type": wave_type,
            "direction": direction,
            "confidence": confidence,
            "notes": f"Dummy sample {i+1} - {wave_type} wave going {direction}",
            "data_key": i + 1,
            "source": "dummy"
        })
    
    return items

print("✓ Dummy data generation function defined")

## Stratified Splitting Function

In [ ]:
def stratified_split(items, train_ratio=0.8, val_ratio=0.1, seed=42):
    """Perform stratified split by wave_type and direction.
    
    Args:
        items: List of dataset records
        train_ratio: Proportion for training set
        val_ratio: Proportion for validation set
        seed: Random seed for reproducibility
        
    Returns:
        Tuple of (train_items, val_items, test_items)
    """
    rng = random.Random(seed)

    # Group by wave_type and direction for stratification
    buckets = defaultdict(list)
    for r in items:
        key = (r["wave_type"], r["direction"])
        buckets[key].append(r)

    train, val, test = [], [], []

    print(f"Stratifying by {len(buckets)} unique (wave_type, direction) combinations:")
    print("-" * 60)
    
    for key, bucket in buckets.items():
        rng.shuffle(bucket)
        n = len(bucket)
        
        # Calculate split sizes
        n_train = max(1, int(round(n * train_ratio)))
        n_val = max(0, int(round(n * val_ratio)))
        
        # Ensure we don't exceed bucket size
        n_train = min(n_train, n)
        n_val = min(n_val, n - n_train)
        n_test = n - n_train - n_val

        print(f"  {key[0]:12} + {key[1]:5}: {n:3} total → {n_train:3} train, {n_val:2} val, {n_test:2} test")
        
        train.extend(bucket[:n_train])
        val.extend(bucket[n_train:n_train + n_val])
        test.extend(bucket[n_train + n_val:])

    # Final shuffle to mix stratification groups
    rng.shuffle(train)
    rng.shuffle(val)
    rng.shuffle(test)
    
    return train, val, test

print("✓ Stratified splitting function defined")

## Load Data and Create Splits

In [ ]:
# Set seed for reproducibility
set_seed(CONFIG["seed"])

# Try to load the real dataset index
items = read_jsonl(CONFIG["real_index"])

if not items and CONFIG["create_dummy_if_missing"]:
    print(f"⚠️ Real dataset index not found at {CONFIG['real_index']}")
    print(f"Creating dummy data for demonstration...")
    items = create_dummy_dataset(CONFIG["dummy_samples"])
    print(f"✓ Created {len(items)} dummy samples")
elif items:
    print(f"✓ Loaded {len(items)} items from {CONFIG['real_index']}")
else:
    raise ValueError(f"No data found and dummy creation disabled")

if len(items) == 0:
    raise ValueError("No data available for splitting")

In [ ]:
# Analyze data distribution before splitting
df_full = pd.DataFrame(items)

print("\nDataset overview before splitting:")
print("=" * 40)
print(f"Total samples: {len(df_full)}")

print(f"\n📏 Height statistics:")
height_stats = df_full['height_meters'].describe()
for stat, value in height_stats.items():
    print(f"  {stat}: {value:.3f}")

print(f"\n🌊 Wave type distribution:")
wave_type_counts = df_full['wave_type'].value_counts()
for wt, count in wave_type_counts.items():
    percentage = count / len(df_full) * 100
    print(f"  {wt}: {count} ({percentage:.1f}%)")

print(f"\n🧭 Direction distribution:")
direction_counts = df_full['direction'].value_counts()
for direction, count in direction_counts.items():
    percentage = count / len(df_full) * 100
    print(f"  {direction}: {count} ({percentage:.1f}%)")

# Show stratification groups
strat_keys = df_full.groupby(['wave_type', 'direction']).size()
print(f"\n📊 Stratification groups (wave_type, direction):")
for (wt, direction), count in strat_keys.items():
    print(f"  {wt} + {direction}: {count} samples")

In [ ]:
# Perform stratified split
print(f"\nPerforming stratified split...")
print(f"Target ratios: Train={CONFIG['train_ratio']}, Val={CONFIG['val_ratio']}, Test={CONFIG['test_ratio']}")
print()

train, val, test = stratified_split(
    items, 
    train_ratio=CONFIG['train_ratio'], 
    val_ratio=CONFIG['val_ratio'], 
    seed=CONFIG['seed']
)

print(f"\n✅ Split results:")
print(f"Train: {len(train)} samples ({len(train)/len(items)*100:.1f}%)")
print(f"Val: {len(val)} samples ({len(val)/len(items)*100:.1f}%)")
print(f"Test: {len(test)} samples ({len(test)/len(items)*100:.1f}%)")
print(f"Total: {len(train) + len(val) + len(test)} samples")

# Verify no data loss
assert len(train) + len(val) + len(test) == len(items), "Data loss during splitting!"

## Save Splits

In [ ]:
# Create output directory and save splits
ensure_dir(CONFIG["out_dir"])

train_path = os.path.join(CONFIG["out_dir"], "train.jsonl")
val_path = os.path.join(CONFIG["out_dir"], "val.jsonl")
test_path = os.path.join(CONFIG["out_dir"], "test.jsonl")

write_jsonl(train, train_path)
write_jsonl(val, val_path)
write_jsonl(test, test_path)

print(f"✓ Saved splits to {CONFIG['out_dir']}/")
print(f"  Train: {train_path} ({len(train)} samples)")
print(f"  Val: {val_path} ({len(val)} samples)")
print(f"  Test: {test_path} ({len(test)} samples)")

## Split Analysis and Validation

In [ ]:
# Convert splits to DataFrames for analysis
df_train = pd.DataFrame(train)
df_val = pd.DataFrame(val)
df_test = pd.DataFrame(test)

# Add split labels for combined analysis
df_train['split'] = 'train'
df_val['split'] = 'val'
df_test['split'] = 'test'

df_combined = pd.concat([df_train, df_val, df_test], ignore_index=True)

print("Split DataFrames created for analysis")
print(f"Combined DataFrame shape: {df_combined.shape}")

In [ ]:
# Analyze distribution preservation across splits
print("Distribution Analysis Across Splits:")
print("=" * 50)

print("\n1. Wave Type Distribution (% within each wave type):")
wave_type_dist = pd.crosstab(df_combined['wave_type'], df_combined['split'], normalize='index') * 100
print(wave_type_dist.round(1))

print("\n2. Direction Distribution (% within each direction):")
direction_dist = pd.crosstab(df_combined['direction'], df_combined['split'], normalize='index') * 100
print(direction_dist.round(1))

print("\n3. Combined (Wave Type, Direction) Distribution:")
df_combined['strat_key'] = df_combined['wave_type'] + '_' + df_combined['direction']
combined_dist = pd.crosstab(df_combined['strat_key'], df_combined['split'], normalize='index') * 100
print(combined_dist.round(1))

# Check for good stratification (should be close to target ratios)
target_train, target_val, target_test = CONFIG['train_ratio']*100, CONFIG['val_ratio']*100, CONFIG['test_ratio']*100
print(f"\nTarget ratios: Train={target_train:.1f}%, Val={target_val:.1f}%, Test={target_test:.1f}%")

# Calculate average deviations
train_dev = abs(wave_type_dist['train'] - target_train).mean()
val_dev = abs(wave_type_dist['val'] - target_val).mean()
test_dev = abs(wave_type_dist['test'] - target_test).mean()

print(f"Average deviations from target ratios:")
print(f"  Train: ±{train_dev:.1f}%")
print(f"  Val: ±{val_dev:.1f}%")
print(f"  Test: ±{test_dev:.1f}%")

## Visualization of Split Distributions

In [ ]:
# Visualize split distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Height distributions by split
split_data = [("Train", df_train), ("Val", df_val), ("Test", df_test)]
colors = ['skyblue', 'lightcoral', 'lightgreen']

for i, (split_name, split_df) in enumerate(split_data):
    axes[0, i].hist(split_df['height_meters'], bins=15, alpha=0.7, 
                   edgecolor='black', color=colors[i])
    axes[0, i].set_title(f'{split_name} Set - Height Distribution')
    axes[0, i].set_xlabel('Height (meters)')
    axes[0, i].set_ylabel('Frequency')
    axes[0, i].axvline(split_df['height_meters'].mean(), color='red', linestyle='--', 
                      label=f'Mean: {split_df["height_meters"].mean():.2f}m')
    axes[0, i].legend()
    axes[0, i].grid(True, alpha=0.3)

# Wave type distributions
wave_type_counts = [df_train['wave_type'].value_counts(), 
                   df_val['wave_type'].value_counts(), 
                   df_test['wave_type'].value_counts()]

for i, (split_name, counts) in enumerate(zip(["Train", "Val", "Test"], wave_type_counts)):
    bars = axes[1, i].bar(counts.index, counts.values, color=colors[i], alpha=0.7)
    axes[1, i].set_title(f'{split_name} Set - Wave Type Distribution')
    axes[1, i].set_xlabel('Wave Type')
    axes[1, i].set_ylabel('Count')
    axes[1, i].tick_params(axis='x', rotation=45)
    
    # Add count labels on bars
    for bar, count in zip(bars, counts.values):
        axes[1, i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                       str(count), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Additional visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Direction distribution across splits
direction_counts_by_split = df_combined.groupby(['direction', 'split']).size().unstack(fill_value=0)
direction_counts_by_split.plot(kind='bar', ax=axes[0, 0], color=colors)
axes[0, 0].set_title('Direction Distribution Across Splits')
axes[0, 0].set_xlabel('Direction')
axes[0, 0].set_ylabel('Count')
axes[0, 0].legend(title='Split')
axes[0, 0].tick_params(axis='x', rotation=0)

# Confidence distribution across splits
conf_counts_by_split = df_combined.groupby(['confidence', 'split']).size().unstack(fill_value=0)
conf_counts_by_split.plot(kind='bar', ax=axes[0, 1], color=colors)
axes[0, 1].set_title('Confidence Distribution Across Splits')
axes[0, 1].set_xlabel('Confidence Level')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend(title='Split')
axes[0, 1].tick_params(axis='x', rotation=0)

# Height box plots by split
split_names = ['train', 'val', 'test']
height_data = [df_combined[df_combined['split'] == split]['height_meters'].values 
               for split in split_names]

box_plot = axes[1, 0].boxplot(height_data, labels=['Train', 'Val', 'Test'], patch_artist=True)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

axes[1, 0].set_title('Height Distribution by Split (Box Plot)')
axes[1, 0].set_xlabel('Split')
axes[1, 0].set_ylabel('Height (meters)')
axes[1, 0].grid(True, alpha=0.3)

# Split size pie chart
split_sizes = [len(train), len(val), len(test)]
split_labels = [f'Train\n({len(train)})', f'Val\n({len(val)})', f'Test\n({len(test)})']

axes[1, 1].pie(split_sizes, labels=split_labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Split Size Distribution')

plt.tight_layout()
plt.show()

## Statistical Tests for Split Quality

In [ ]:
# Statistical tests for distribution similarity
print("Statistical Tests for Split Quality:")
print("=" * 50)

# Kolmogorov-Smirnov test for height distributions
ks_train_val = ks_2samp(df_train['height_meters'], df_val['height_meters'])
ks_train_test = ks_2samp(df_train['height_meters'], df_test['height_meters'])
ks_val_test = ks_2samp(df_val['height_meters'], df_test['height_meters'])

print(f"\n📊 Height Distribution Similarity (KS test p-values):")
print(f"  Train vs Val: {ks_train_val.pvalue:.4f}")
print(f"  Train vs Test: {ks_train_test.pvalue:.4f}")
print(f"  Val vs Test: {ks_val_test.pvalue:.4f}")
print("  (Higher p-values indicate more similar distributions)")

# Chi-square test for categorical distributions
wave_type_contingency = pd.crosstab(df_combined['wave_type'], df_combined['split'])
chi2_wt, p_wt, _, _ = chi2_contingency(wave_type_contingency)

direction_contingency = pd.crosstab(df_combined['direction'], df_combined['split'])
chi2_dir, p_dir, _, _ = chi2_contingency(direction_contingency)

print(f"\n📊 Categorical Distribution Similarity (Chi-square test p-values):")
print(f"  Wave type across splits: {p_wt:.4f}")
print(f"  Direction across splits: {p_dir:.4f}")
print("  (Higher p-values indicate more similar distributions)")

# Interpretation
print(f"\n💡 Interpretation:")
if all(p > 0.05 for p in [ks_train_val.pvalue, ks_train_test.pvalue, ks_val_test.pvalue]):
    print("  ✓ Height distributions are statistically similar across splits")
else:
    print("  ⚠️ Some height distributions differ significantly between splits")

if p_wt > 0.05 and p_dir > 0.05:
    print("  ✓ Categorical distributions are well-balanced across splits")
else:
    print("  ⚠️ Some categorical imbalances detected across splits")

## Detailed Split Summary

In [ ]:
# Detailed split summary
print("Detailed Split Summary:")
print("=" * 60)

for split_name, split_df in [("TRAINING", df_train), ("VALIDATION", df_val), ("TEST", df_test)]:
    print(f"\n{split_name} SET:")
    print(f"  📊 Samples: {len(split_df)} ({len(split_df)/len(items)*100:.1f}% of total)")
    
    print(f"  📏 Height statistics:")
    print(f"    Range: {split_df['height_meters'].min():.2f}m - {split_df['height_meters'].max():.2f}m")
    print(f"    Mean: {split_df['height_meters'].mean():.2f}m ± {split_df['height_meters'].std():.2f}m")
    print(f"    Median: {split_df['height_meters'].median():.2f}m")
    
    print(f"  🌊 Wave types: {dict(split_df['wave_type'].value_counts())}")
    print(f"  🧭 Directions: {dict(split_df['direction'].value_counts())}")
    print(f"  ⚖️ Confidence levels: {dict(split_df['confidence'].value_counts())}")
    
    # Unique stratification groups in this split
    strat_groups = split_df.groupby(['wave_type', 'direction']).size()
    print(f"  📋 Stratification groups: {len(strat_groups)} unique combinations")

## Validation Checks

In [ ]:
# Comprehensive validation checks
print("Split Validation Checks:")
print("=" * 40)

# Check 1: No overlap between splits
train_paths = set(r['image_path'] for r in train)
val_paths = set(r['image_path'] for r in val)
test_paths = set(r['image_path'] for r in test)

overlap_train_val = train_paths & val_paths
overlap_train_test = train_paths & test_paths
overlap_val_test = val_paths & test_paths

if not overlap_train_val and not overlap_train_test and not overlap_val_test:
    print("✓ No overlap between splits - data integrity maintained")
else:
    print("✗ Overlap detected between splits!")
    if overlap_train_val:
        print(f"  Train-Val overlap: {len(overlap_train_val)} samples")
    if overlap_train_test:
        print(f"  Train-Test overlap: {len(overlap_train_test)} samples")
    if overlap_val_test:
        print(f"  Val-Test overlap: {len(overlap_val_test)} samples")

# Check 2: Total count preservation
total_split = len(train) + len(val) + len(test)
if total_split == len(items):
    print(f"✓ All {len(items)} samples accounted for in splits")
else:
    print(f"✗ Sample count mismatch: {total_split} in splits vs {len(items)} original")

# Check 3: Minimum samples per class in each split
min_samples_per_class = 1
for split_name, split_df in [("train", df_train), ("val", df_val), ("test", df_test)]:
    wt_counts = split_df['wave_type'].value_counts()
    dir_counts = split_df['direction'].value_counts()
    
    if wt_counts.min() >= min_samples_per_class and dir_counts.min() >= min_samples_per_class:
        print(f"✓ {split_name.capitalize()} split has sufficient samples per class")
    else:
        print(f"⚠️ {split_name.capitalize()} split has classes with few samples:")
        if wt_counts.min() < min_samples_per_class:
            sparse_wt = wt_counts[wt_counts < min_samples_per_class]
            print(f"    Wave types: {sparse_wt.to_dict()}")
        if dir_counts.min() < min_samples_per_class:
            sparse_dir = dir_counts[dir_counts < min_samples_per_class]
            print(f"    Directions: {sparse_dir.to_dict()}")

# Check 4: Split ratio validation
actual_ratios = {
    'train': len(train) / len(items),
    'val': len(val) / len(items),
    'test': len(test) / len(items)
}

target_ratios = {
    'train': CONFIG['train_ratio'],
    'val': CONFIG['val_ratio'],
    'test': CONFIG['test_ratio']
}

print(f"\n📊 Split ratio validation:")
ratio_ok = True
for split in ['train', 'val', 'test']:
    actual = actual_ratios[split]
    target = target_ratios[split]
    diff = abs(actual - target)
    
    if diff < 0.05:  # Within 5% tolerance
        print(f"  ✓ {split.capitalize()}: {actual:.3f} (target: {target:.3f}, diff: {diff:.3f})")
    else:
        print(f"  ⚠️ {split.capitalize()}: {actual:.3f} (target: {target:.3f}, diff: {diff:.3f})")
        ratio_ok = False

if ratio_ok:
    print("✓ All split ratios are within acceptable tolerance")
else:
    print("⚠️ Some split ratios deviate significantly from targets")

print(f"\n✅ Splits validation complete!")

## Export Split Statistics

In [ ]:
# Save comprehensive split statistics
split_stats = {
    "config": CONFIG,
    "dataset_info": {
        "total_samples": len(items),
        "source": "real_data" if not CONFIG["create_dummy_if_missing"] or os.path.exists(CONFIG["real_index"]) else "dummy_data",
        "stratification_groups": len(strat_keys)
    },
    "split_sizes": {
        "train": len(train),
        "val": len(val),
        "test": len(test)
    },
    "split_ratios": {
        "actual": actual_ratios,
        "target": target_ratios
    },
    "height_stats": {
        "train": df_train['height_meters'].describe().to_dict(),
        "val": df_val['height_meters'].describe().to_dict(),
        "test": df_test['height_meters'].describe().to_dict()
    },
    "wave_type_distribution": {
        "train": df_train['wave_type'].value_counts().to_dict(),
        "val": df_val['wave_type'].value_counts().to_dict(),
        "test": df_test['wave_type'].value_counts().to_dict()
    },
    "direction_distribution": {
        "train": df_train['direction'].value_counts().to_dict(),
        "val": df_val['direction'].value_counts().to_dict(),
        "test": df_test['direction'].value_counts().to_dict()
    },
    "confidence_distribution": {
        "train": df_train['confidence'].value_counts().to_dict(),
        "val": df_val['confidence'].value_counts().to_dict(),
        "test": df_test['confidence'].value_counts().to_dict()
    },
    "cross_tabulation": {
        "wave_type_vs_direction": pd.crosstab(df_full['wave_type'], df_full['direction']).to_dict()
    },
    "statistical_tests": {
        "ks_tests": {
            "train_vs_val": ks_train_val.pvalue,
            "train_vs_test": ks_train_test.pvalue,
            "val_vs_test": ks_val_test.pvalue
        },
        "chi2_tests": {
            "wave_type": p_wt,
            "direction": p_dir
        }
    },
    "validation_results": {
        "no_overlap": len(overlap_train_val) == 0 and len(overlap_train_test) == 0 and len(overlap_val_test) == 0,
        "count_preserved": total_split == len(items),
        "ratios_within_tolerance": ratio_ok
    }
}

# Save statistics
stats_path = os.path.join(CONFIG["out_dir"], "split_statistics.json")
with open(stats_path, 'w', encoding='utf-8') as f:
    json.dump(split_stats, f, indent=2, ensure_ascii=False)

# Save summary CSV
summary_df = pd.DataFrame([
    {'split': 'train', 'samples': len(train), 'ratio': actual_ratios['train']},
    {'split': 'val', 'samples': len(val), 'ratio': actual_ratios['val']},
    {'split': 'test', 'samples': len(test), 'ratio': actual_ratios['test']}
])
summary_df.to_csv(os.path.join(CONFIG["out_dir"], "split_summary.csv"), index=False)

print(f"✅ Statistics and summaries saved:")
print(f"  📊 Detailed statistics: {stats_path}")
print(f"  📋 Summary CSV: {os.path.join(CONFIG['out_dir'], 'split_summary.csv')}")

print(f"\n🎯 Final Summary:")
print(f"  Total samples: {len(items)}")
print(f"  Stratification groups: {len(strat_keys)}")
print(f"  Train: {len(train)} samples ({actual_ratios['train']:.1%})")
print(f"  Val: {len(val)} samples ({actual_ratios['val']:.1%})")
print(f"  Test: {len(test)} samples ({actual_ratios['test']:.1%})")
print(f"  Output directory: {CONFIG['out_dir']}")

## Summary

This notebook provides a complete, standalone system for creating stratified dataset splits:

### ✅ **What we accomplished:**
1. **Flexible Data Loading**: Works with real data or creates dummy data for demonstration
2. **Stratified Splitting**: Ensures balanced representation across wave types and directions
3. **Quality Validation**: Comprehensive checks for data integrity and distribution balance
4. **Statistical Analysis**: KS tests and Chi-square tests for distribution similarity
5. **Rich Visualizations**: Distribution plots, box plots, and comparative analysis
6. **Comprehensive Export**: Multiple output formats with detailed statistics

### 🔧 **Key Features:**
- **Stratified Sampling**: Maintains class balance across all splits
- **Robust Validation**: Multiple checks for split quality and integrity
- **Statistical Testing**: Quantitative validation of distribution similarity
- **Flexible Configuration**: Easy to adjust ratios and parameters
- **Dummy Data Support**: Works without real data for testing and development

### 📊 **Split Quality Metrics:**
- **No Data Leakage**: Verified no overlap between train/val/test sets
- **Balanced Representation**: Each class appears in all splits proportionally
- **Statistical Similarity**: Distributions are statistically similar across splits
- **Ratio Compliance**: Actual ratios match target ratios within tolerance

### 🚀 **Usage:**
- **With Real Data**: Point to your dataset index JSONL file
- **Without Real Data**: Automatic dummy data generation for testing
- **Customizable**: Adjust split ratios, stratification keys, and validation criteria
- **Production Ready**: Comprehensive validation and error handling

### 📁 **Output Files:**
- `train.jsonl`, `val.jsonl`, `test.jsonl`: Split dataset files
- `split_statistics.json`: Comprehensive statistics and validation results
- `split_summary.csv`: Quick summary table

**This notebook is completely standalone and ensures high-quality, balanced dataset splits for machine learning!**